In [ ]:
import numpy as np # linear algebra 
from math import log, sqrt # neperian logarithm, square root
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import nltk


## Read the train
train_df = pd.read_csv("../input/train.csv")

authors = {'EAP':0,'HPL':1,'MWS':2}
authors_rev = ['EAP','HPL','MWS']

# A step-by-step mathematical approach to Spooky | [English]/[French]

It is often regretted that computer scientists are not always very rigorous.
They manipulate mathematics ... but in their own way and sometimes make strange mishmash with!

We aim to show that one can harmoniously combine mathematics &amp; computer science; that is to say, at the same time to make the computer science properly and the mathematics usefull!

Here is the course of this tutorial:
- we will begin with a short introduction about this area straddling the linguistics and the satistics which aims to identify the authors of texts (and so which is at the heart of our competition Soopky) : * ** the stylometry ** *
- we will then think about Spooky's notation system: the "log loss" (or * cross-entropy *) notation: 
$-\frac{1}{N}\sum_{i = 1}^N\sum_{j = 1}^3y_{ij}\log(p_ {ij}).$
Assuming that we already have the program that transforms a sentence into a vector of $ \mathbb R ^ d $, we will see what are the quantities $p_{ij}^{opt}$ to return to minimize the expectation of the "log loss"
- after, we will present a very simple algorithm: the k-nearest-neighbors (kNN) algorithm. This algorithm has the mathematical property of asymptotically returning (when $ N \to \infty $) the optimal probabilities $ p_ {ij} ^ {opt} $ that we are looking for.
- finally, we'll show some examples of programs (with the code python!) to transform our sentences into vectors. At first, some common examples (length of sentences, length of words, presence of some typical words, etc ...) Then a more elaborate example which calculates the log-likelihood of a sentence for each of the authors from their vocabulary. Finally, we will propose an example of an even more elaborate kernel using the library "nltk": it allows to transform a sentence into a tree from its grammatical structure.
- we will devote the last part to the visualization of the data. We will present two algorithms: that of the Principal Component Analysis (PCA) which consists of an orthogonal transformation and that of the "t-SNE" which decreases the dimension of the space $ \mathbb R ^ d $ while seeking to report the distances of the original space in the new space.


<ul>
  <li> <a href="#aintroduction"> Introduction to stylometry </a></li>
  <li> <a href="#alogloss"> What to look for to minimize log loss? </a></li>
  <li> <a href="#akppv"> The k-nearest-neighbors algorithm (which asymptotically minimizes log loss) </a></li>
  <li> <a href="#aexemples"> Some examples of features </a>
      <ul style = "list-style-type: circle">
      <li> <a href="#aexemples_courants"> Classic examples of stylometry </a></li>
      <li> <a href="#avoca_auteurs"> Calculating log-likelihood for each author by vocabulary </a></li>
      <li> <a href="#akernel"> Kernel on the grammatical structure of sentences </a></li>
      </ul>
  </li>
  <li> <a href="#avisualisation"> Data Visualization </a>
        <ul style = "list-style-type: circle">
      <li> <a href="#aACP"> Principal Component Analysis (PCA) </a> </li>
      <li> <a href="#atSNE"> t-SNE </a> </li>
      </ul>
  </li>
   <li> <a href="#aresultats"> Submission and results (score : 0.40)</a></li>
</ul>

**NB**: we ask the reader to kindly excuse the many faults of English disseminated throughout this tutorial; English is not my mother tongue.
Moreover, the French speaking reader can find a French version of this tutorial at the end: <a href="#francais">french version</a>.

<div id="aintroduction" />

# Introduction to stylometry
Kaggle offers us, with the help of a training corpus, to find the authors (who are three in number: Edgar A. Poe, Howard P. Lovecraft and Mary Shelley) of about twenty thousand sentences.

What Kaggle asks us to do is named: *** stylometry ***.

Stylometry is this *art* (taken in its Greek sense of *τέχνη*) at the crossroads of statistics and linguistics whose purpose is to produce from text an *information* (which can be treated statistically ) reporting on the *style* of these; that is to say, which would characterize - ideally, this means ... - at the same time its author, but also its kind, its time, etc ...
More practically, these statistics generally relate to the vocabulary used (which can be refined by identifying the meaning in which the author uses such a word), the grammatical categories (nouns / pronouns, adjectives, adverbs, verbs ... at what time / mode?) as well as the grammatical structure of the sentences, punctuation, etc ...

It must be understood that to make good stylometry, it is necessary to play in both ways:
- that of linguistics by proposing criteria (features) that are most relevant to isolate the style of an author,
- that of statistics by using adapted mathematical and computer tools.

Throughout this tutorial, we'll try to take up these two aspects of stylometry by always being as rigorous as possible and justifying (as far as possible) everything we do.

But first, let's look at Spooky's evaluation system:

<div id="alogloss" />
# What to look for to minimize log loss?

First we will see why the evaluation system "logloss" has been chosen and what are the mathematical values to look for to minimize it in expectation.
From these values to look for, we will deduce a suitable algorithm (that is to say that we justify from a mathematical point of view) for the computation of the probabilities $ p_ {ij} $ to return. This is the agorithm of the k-nearest-neighbors.

The notation system that has been chosen for Spooky is the "log loss" (or *cross entropy*):
        
$$ log loss = - \frac{1}{N} \sum_{\substack {i \in \{1, ..., N \} \\ j \in \{EAP, HPL, MWS \}}} y_{ij} \log (p_ {ij}) $$
where $ p_ {ij} $ is the estimate: the probability according to our program that the text $ i $ is of the author $ j $, $ y_{ij} $ is the solution: it equals $ 1 $ when the text $ i $ is of the author $ j $, $ 0 $ otherwise.

The goal is to minimize this log loss.

We will now formalize a little bit the problem from a mathematical point of view in order to compute the expected log loss and to find the optimal values $ p_{ij}^{opt} $ to return to achieve the minimal expectation.

Suppose we already have the first part of the program: the one that transforms a sentence into a computer-processable "information". In this case we will assume that this information is a vector of $ \mathbb R ^ d $ and that they are all independently fetched according to the same law $ X $ admitting a compact $ f $ density. It is also assumed that this law $ \mathcal L (X) $ can be broken down into two stages:

- first, we choose the author of the sentence according to a law of "Bernoulli with three issues": 'EAP' for Edgar A. Poe, 'HPL' for Howard P. Lovecraft and 'MWS' for Mary Shelley. We note $ \mathcal B (\lambda_1, \lambda_2, \lambda_3) $ this law (with $ \sum \lambda_j = 1 $); $ \lambda_j $ being the proportion of $ j $ author's sentences.
- According to the author $ j $ which is chosen in the first step, our sentence is drawn according to a law of density $ f_j $ on $ \mathbb R ^ d $.

![shema_bernouilli](http://bourg-la-reine-echecs.fr/telechargements/shema_bernoulli_auteurs_corrige.png)

So we have that: $ f = \sum \lambda_j f_j $.

On the training corpus, we get:

In [ ]:
nb_texts = len(train_df)
lambdas = (train_df.author.value_counts()/nb_texts).to_dict()
print("Lambdas :",lambdas)

- $ \lambda_1 = 0.403493538995863 $
- $ \lambda_2 = 0.287808366106543 $
- $ \lambda_3 = 0.308698094897594 $

The attentive reader may have noticed that the sample file "sample\_submission" for each sentence precisely returns the probabilities $ (\lambda_1, \lambda_2, \lambda_3) $ ... it's no coincidence !! It is the return that minimizes the "log loss" with the worst program: the one that does not distinguish anything and that sends all the sentences on the null vector ($ \mathcal L (X) = \delta_0 $ where $ \delta_0 $ is the measure of "dirac" at the point $ 0 $).
This follows from a classical result of information theory: if $ (\lambda_j)_j $ and $ (p_j)_j $ are two discrete probability measures, then cross entropy:
$$ H_\lambda (p) = \mathbb E [log loss] = - \sum_j \lambda_j \log (p_j) $$
is minimized when $ p_j = \lambda_j $ for all $ j $.

We will now generalize this result!

Given a sentence transformed by our program into a $ x $ vector (according to the $ X $ law), the best solution (in the sense that it is the one that minimizes the expectation of the final "log loss" score) of Probabilities to return is the triplet:

$$ p_1 ^ {opt} (x) = \frac{\lambda_1 f_1 (x)} {\sum_j \lambda_j f_j (x)}, \ \ \ \
p_2 ^ {opt} (x) = \frac{\lambda_2 f_2 (x)}{\sum_j \lambda_j f_j (x)}, \ \ \ \
p_3 ^ {opt} (x) = \frac{\lambda_3 f_3 (x)} {\sum_j \lambda_j f_j (x)}. $$

Good! now that it has been said, there is more to roll up the sleeves!

The expectation of the "log loss" rating is given in this case by the following formula:

$$ \mathbb E [log loss] = - \sum_j {\lambda_j \int_ {\mathbb R ^ d} \log (p_j (x)) f_j (x) \mathrm d x} $$
where $ p_j (x) $ is the probability that we return for the author $ j $ when we receive the vector $ x $; it is our estimation that the sentence that has been transformed into the vector $ x $ is of the author $ j $ (we want to show that $ p_j ^ {opt} (x) = \frac {\lambda_j f_j (x )} {\sum_k \lambda_k f_k (x)} $ is indeed the optimal solution).

In general, the program will return a probability that can be written in the form:
$$ q_j (x) = \frac {\lambda_j g_j (x)} {\sum_k \lambda_k g_k (x)} $$
where $ g_j $ is a positive continuous function (which is assumed to be at least as large as $ f_j $ because if there exists $ x $ such that $ q_j (x) = 0 $ and $ f_j ( x)> 0 $, then $ \mathbb P (logloss = \infty)> 0 $ ...).
So that the $ g_j $ will "lose" their weight outside the support of $ f $ (that is, if $ f (x) = 0 $, $ x $ has no chance to appear, so we do not care about the value of $ g_j $ in these points), we can also assume that $ g_j $ are density functions (that is, $ \int _{\mathbb R ^ d} g_j (x) dx = 1 $).

We can take, for example, $ g_j = \frac {1} {\lambda_j} f_j $ to disregard the difference in the number of sentences between authors (here, $g_j$ are not density functions since their integral does not equal $ 1 $, but one could come back to it as said just before).
If we want "extreme" probabilities (close to $ 0 $ or $ 1 $), we can ask: $ g_j = f_j ^ 2 \times \frac {1} {\int f_j ^ 2} $.
On the contrary, if one wishes to "qualify" these probabilities, one will ask: $ g_j = \sqrt {f_j} \times \frac {1} {\int \sqrt f_j} $.

Now let us show that the best choice for minimizing log loss is: $ p_j ^ {opt} = \frac {\lambda_j f_j (x)} {\sum_k \lambda_k f_k (x)} $. Let us note $ log loss ^ {opt} $ the score obtained with this last choice and $ log loss ^ {q_j} $ the score obtained with the probabilities $ q_j $ defined using the functions $ g_j $.
We then have:

\begin{align}
\mathbb E[log loss^{opt}] - \mathbb E[log loss^{q_j}] & = - \sum_j{\lambda_j \int_{\mathbb R ^d}\log(p_j^{opt})f_j(x) \mathrm d x} - \sum_j{\lambda_j \int_{\mathbb R ^d}\log(q_j)f_j(x) \mathrm d x}\\
 & = - \sum_j{\lambda_j \int_{\mathbb R^d}(\log(p_j^{opt})-\log(q_j))f_j(x) \mathrm d x} \\
 & = - \sum_j{\lambda_j \int_{\mathbb R^d}\log(\frac{\lambda_j f_j(x)}{\lambda_j g_j(x)}\frac{\sum_k \lambda_k g_k(x)}{\sum_k \lambda_k f_k(x)})f_j(x) \mathrm d x} \\
 & = - \sum_j{\lambda_j \int_{\mathbb R ^d}\log(\frac{ f_j(x)}{ g_j(x)})f_j(x) \mathrm d x} + {\int_{\mathbb R ^d}\log(\frac{\sum_k \lambda_k f_k(x)}{\sum_k \lambda_k g_k(x)})(\sum_j \lambda_j f_j(x))\mathrm d x} \\
 & = - \sum_j \lambda_j D_{KL}(f_j || g_j) + D_{KL}(\sum_k \lambda_k f_k || \sum_k \lambda_k g_k)\\
 & \text{(where $ D_ {KL} $ denotes the Kullback-Leibler divergence ... we'll get back to it!)} \\
 & \le 0 \text{ by the convexity property of $ D_ {KL} $ in the function pairs;}\\
 & \text{here, there is $ 3 $: $ (f_1, g_1) $, $ (f_2, g_2) $ and $ (f_3, g_3) $. We conclude that $ p_j ^ {opt} $ is optimal:}\\
 \mathbb E[log loss^{opt}] & \le \mathbb E[log loss^{q_j}]
\end{align}

The Kullback-Leibler divergence between two densities $ p $ and $ q $ on $ \mathbb R ^ d $ is given by the following formula:
$ D_ {KL} (p || q) = \int_{\mathbb R ^ d} \log (\frac{p (x)}{q (x)}) p (x) \mathrm d x $. To see it quickly, in information theory, it is the average amount of additional bits that will be needed to code from the source $ q $ an optimal code of the source $p$.
We will talk again about the divergence of Kullback-Leibler in the part devoted to the "t-SNE" (algorithm that works precisely on a minimization of this same divergence during the change of dimension of the data space).


So we took the first step boldly (our point being to show that we can do IT cleanly and rigorously!): We now know that once we have our program that turns sentences into vectors, the amount to look for (and return to Kaggle) is:
$$ p_j ^ {opt} (x) = \frac {\lambda_j f_j (x)}{\sum_k \lambda_k f_k (x)}. $$

.



<div id="akppv" />

# K-nearest-neighbors (to find the optimal $ p_ {ij} ^ {opt} $ probabilities)

We will show a way to recover these probabilities. Oh ! it is certainly not an extraordinary algorithm since it is the nearest-neighbors algorithm.
Nevertheless, it has the advantage of asymptotically giving (when $ N $ the number of texts tends to infinity) the best possible result. Other algorithms often used are, for example, the hypothesis of a Gaussian distribution of data (this is the case of classifiers naive Bayes) which give good results on small samples (since they are based on the computation of expectations and covariances of the variables, which requires much less data than to estimate a density) but these algorithms cease to give good results as soon as the learning base is sufficient.

Here is the algorithm of K-nearest-neighbors:

- We have a learning base $ \{(x_1, y_1), \ ... \ , \ (x_N, y_N) \} $ containing the $ N $ vectors $ x_1, ..., x_N \in \mathbb R ^ d $ formed from $ N $ sentences with their respective author $ y_1, ..., y_N \in \{1,2,3 \} $. We also choose a distance $ d $ on $ \mathbb R ^ d $ (usually the Euclidean distance, but this is not obligatory, the asymptotic results will remain true for any distance: we say that the operator of the kNN is *universally* consistent).
- We choose an integer $ k $ depending on $ N $ which must be small in front of $ N $ (we want: $ k / N \underset {N \to \infty} {\longrightarrow} 0 $) but still big enough (it must be that: $ k \underset {N \to \infty} {\longrightarrow} \infty $).
- Now, each time we receive a new vector $ x $, we consider the set $kNN$ of the $k$ indices  of the points the closer to $x$:

$$kNN = \{i_1 <..<i_k \ | \ \forall \ j \notin \{i_1, ..., i_N \}, \forall \ i \in \{i_1, ..., i_N \},  \ d (x, x_i) \le d (x , x_j) \}$$


- Finally, we return probabilities $ \hat p_j (x) $ proportional to the number of more-near-neighbors who are of the author $ j $. So :
$$ \text {For $ j \in \{1,2,3 \} $, } \hat p_j (x) = \frac {1} {k} \# \{i \in kNN \ | \ y_i = j \} $$


We have therefore presented the very simple $ k $-NN algorithm which returns $ \hat p_j (x) $ estimates of $ p_j ^ {opt} (x) $. It checks the following "weak consistency" property:
$$ \text{If $ k \underset {N \to \infty} {\longrightarrow} \infty $ and $ k / N \underset {N \to \infty} {\longrightarrow} 0 $, then:} \ \forall \ \epsilon> 0, \ \mathbb P (| \hat p_j - p_j ^ {opt} |> \epsilon) \underset {N \to \infty} {\longrightarrow} 0 $$

Asymptotically, we thus find empirically the value of $ p_j ^ {opt} $. This does not tell us anything about the speed of convergence. There are refinements of this algorithm (for example by using proximity graphs) which make it possible to improve this speed of convergence.


<div id="aexemples" />
# Examples of features

<div id="aexemples_courants" />
## - Current Examples
TODO

<div id="avoca_auteurs" />
## - Example of vocabulary usage

The examples we saw in the previous section are common examples of stylometry. They are also very general examples in that they do not require the presence of a training corpus: they could equally well be used in the case of *unsupervised classification* (and not only *supervised* as this is the case here).

We'll explain now an example of a function that uses the vocabulary used in the training set. Given a sentence, this function returns a $ 3 $ size vector containing the log-likelihood of using that vocabulary for each of the $ 3 $ authors.

Let's split our training corpus in two parts: 
- a training set to build the dictionary ($95\%$ of the data)
- a test set that will allow us to visualize the results ($5\%$ of the data).

In [ ]:
N_train = len(train_df)

# cutting of the train set
cut = round(N_train * 0.95)

train = pd.DataFrame(train_df, index=range(cut))
test = pd.DataFrame(train_df, index=range(cut,N_train))


Let's start by building a dictionary containing all the words appearing in the training corpus. To each entry (a word) in this dictionary we'll associate its occurrences for each of the authors.


In [ ]:
# built from a training corpus a dictionary of used words. For each word, we fill a dictionary that each author associates the number of times this word appeared in the author
def voca_Authors(train):
    voca = {}
    for i,line in train.iterrows():
        words = nltk.word_tokenize(line['text'])
        for word in words:
            word = word.lower()
            if not word in voca:
                voca[word] = {auth:0 for auth in authors}
                voca[word][line['author']] = 1
            else:
                voca[word][line['author']] += 1
    return voca

print('Computation of vocabulary dictionary.')
voca_authors = voca_Authors(train)
print('Dictionary size:',len(voca_authors),'\n20 examples from this dictionary :\n')
for i,word in enumerate(voca_authors):
    if i < 20:
        print(word,voca_authors[word])

All we need know is to understand what has been computed and what is computable.

We will make the hypothesis quite strong (that is to say rather false) that:
- the length of the sentences follows a law independent of the author (one can look at the histograms of the lengths of sentence according to the author with the following address: link and to note that indeed these 3 histograms are quite close) ,
- all the words of a sentence are drawn independently and identically according to a law that depends only on the author (this hypothesis is very strong and very false).

Given those independence hypotheses, the computation of the likelihood for an author who has written the sentence $ s = [w_1, ..., w_k] $ is quite straightforward :

$$ L(w_1, \ ... \ , \ w_k \ | \ author = j) = \prod_ {i = 1} ^ k \mathbb P (\text {author of $ w_i $ = $ j $} \ | \ \text {word = $ w_i $}) $$

Now, let's take the word "of" as an example. We get the following statistics:

In [ ]:
voca_authors['of']

There are $ 8539 + 5568 + 5850 = 19957 $ occurrences of the word "of" in our training corpus. Edgar A. Poe is the author of 8539 of them, that is, a proportion $ x_1=8539/19957=0.43$. 
He is also the author of a proportion $\lambda_1=0.40<x_1$ of all the sentences of the corpus. Since we've been hypothesizing that the law on the length of sentences was author independent, this means that he therefore writes more the word 'of' than his overrepresentativeness in the corpus would allow; that is, $ p_1 : = \mathbb P (\text {author of $ 'of' = 1 $})> 1/3 $.

If we write $ p_j = \mathbb P (\text {author of 'of' = j }) $ and $ x_j $ the proportion of appearances in the training corpus, we get the following relation:
$$ x_j = \frac {\lambda_j p_j} {\sum_k \lambda_k p_k}. $$

We thus know the $ x_j $ and the $ \lambda_j $ and we want to know the $ p_j $ to get our log-likelihoods!

Note that one can very easily compute $ \sum_k \lambda_k p_k $ using :$$ \sum_j \frac {x_j} {\lambda_j} = \sum_j \frac{p_j}{\sum_k  \lambda_k p_k } = \frac{1}{\sum_k \lambda_k p_k }$$
... and thus find the $ p_j $ we're looking for:
$$ p_j = \frac {x_j} {\lambda_j}\left( \sum_k \frac {x_k} {\lambda_k}\right)^{-1} $$

This gets us the log-likelihood for each author, Yay !
$$ \log L (w_1, ..., w_k \ | \ author = j) = \sum_ {i = 1} ^ k \log (p_j (w_i)) $$

In [ ]:
def vect_Voca(sentence, dict = voca_authors, zero = 0.01):
    words = nltk.word_tokenize(sentence)
    ret = [0]*len(authors)
    for word in words:
        word = word.lower()
        if word in dict:
            vect = [0]*len(authors)
            nb_appar = 0
            for auth in dict[word]:
                nb_appar += dict[word][auth]
            for auth in dict[word]:
                vect[authors[auth]] = dict[word][auth]/nb_appar/lambdas[auth] # vect[j] = x_j/lambda_j = p_j / (Sum lambda_j p_j)
            s = 1/sum(vect) # s = (Sum lambda_j p_j) 
            vect = [p_j_div_sum * s for p_j_div_sum in vect] # vect[j] = p_j

            for j,p_j in enumerate(vect):
                if p_j == 0:
                    ret[j] += log(zero)
                else:
                    ret[j] += log(p_j)
    return ret

... and visualize the comet's tail-like results:


In [ ]:
# the matrix of sentences of the corpus of text transformed using vect_Voca
X = []
col = [] # the color according to the author: Poe => red, Lovecraft => green, Shelley => blue
# colours = {'EAP':'rgb(150, 5, 5)','HPL':'rgb(5, 150, 5)','MWS':'rgb(5, 5, 150)'}
colours = {'EAP':'r','HPL':'g','MWS':'b'}
# computation of X, the size matrix N_tests * 3
for i,line in test.iterrows():
    vect = vect_Voca(line['text'])
    X.append(vect)
    col.append(colours[line['author']])
X = np.array(X)

## The display

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt


fig = plt.figure(figsize=(15,15))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(X[:,0], X[:,1], X[:,2], c=col, marker='+')



<div id="akernel" />
## - Example of kernel on the tree structures of sentences
TODO

<div id="avisualisation" />
# Data visualization


<div id="aACP" />
## - The Principal Components Analysis (PCA)
TODO

<div id="atSNE" />
## - The t-SNE
TODO

<div id="aresultats" />
# Submission (score : 0.40)

A simple submission with kNN just on 3 size vectors of vocabulary log-likelihood :

In [ ]:
from sklearn.neighbors import NearestNeighbors

train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

voca_authors = voca_Authors(train)
# la matrice des phrases du corpus de texte transformées à l'aide de vect_Voca
M = [] # the train matrix
Sol = [] # the solutions of the train matrix
X = [] # the test matrix


for i,line in train.iterrows():
    vect = vect_Voca(line['text'])
    M.append(vect)
    Sol.append(line['author'])
M = np.array(M)

for i,line in test.iterrows():
    vect = vect_Voca(line['text'])
    X.append(vect)
X = np.array(X)


# The kNN algorithm
adjunction = 0.025
nb_ppv = 200 # number of nearest neighbors
M_ppv = NearestNeighbors(n_neighbors=nb_ppv)
M_ppv.fit(M)
tab_kneighbors = M_ppv.kneighbors(X, return_distance=False)

Probas = [] # the return

for i,line in test.iterrows():
    p = [0]*3
    kneighbors = tab_kneighbors[i]
    for kneighbor in kneighbors:
        p[authors[Sol[kneighbor]]] += 1
    s = sum(p)
    p = [x/s for x in p]
    Probas.append(p)

# harmonisation of the probabilities
for v in Probas:
    for i,prob in enumerate(v):
        v[i] = (prob + adjunction)/(1+3*adjunction)


submission = pd.read_csv('../input/sample_submission.csv')
submission.loc[:,['EAP', 'HPL', 'MWS']] = Probas
submission.to_csv("Log_likelihoof_on_vocabulary.csv", index=False)
submission.head()

<div id="francais" />
# La version française / the french version :


# Une approche (pas à pas) mathématique de la stylométrie

On déplore souvent que les informaticiens ne soient pas toujours très rigoureux.
Ils manipulent les mathématiques... mais à leur sauce et font parfois de drôles de tambouilles/une drôle de cuisine avec !?

Le but de ce tutoriel est de tenter de montrer qu'une telle situation n'est pas une fatalité : qu'on peut allier harmonieusement mathématiques & informatique ; c'est-à-dire à la fois faire de l'informatique proprement et des mathématiques qui servent à quelque chose !!

Voici donc le déroulé de ce tutoriel :
- on commencera par une courte introduction à propos de ce domaine à cheval sur la linguistique et la satistiqu consistant à identifier les auteurs et qui est au cœur de notre compétition Soopky... j'ai nommé : la **"stylométrie"**
- on réfléchira ensuite au système de notation de Spooky : la note "log loss" (ou *entropie croisée*) : $ -\frac{1}{N}\sum_{i=1}^N\sum_{j=1}^3 y_{ij}\log(p_{ij})$. En supposant que l'on possède déjà le programme qui transforme une phrase en vecteur de $\mathbb R ^d$, on verra quelles sont les quantités $p_{ij}^{opt}$ à renvoyer pour minimiser l'espérance de la "log loss"
- après, on présentera un algorithme très simple : celui des k-plus-proches-voisins. Cet algorithme possède la propriété mathématique de renvoyer asymptotiquement (quand $N \to \infty$) les probabilités optimales $p_{ij}^{opt}$ que l'on recherche.
- enfin, on présentera quelques exemples de programmes (avec le code python !) pour transformer nos phrases en vecteurs. Dans un premier temps, quelques exemples courants (longueur de phrases, longueur des mots, présence de certains mots typiques, etc...) Puis un exemple un peu plus élaboré qui calcule la log-vraisemblance d'une phrase pour chacun des auteurs à partir de leur vocabulaire. Pour finir, on proposera un exemple de kernel encore plus élaboré utilisant la bibliothèque "nltk" : elle permet de transformer une phrase en un arbre à partir de sa structure grammaticale. 
- on consacrera la dernière partie à la visualisation des données. On présentera deux algorithmes : celui de l'Analyse en Composantes Principales qui consiste en un changement de base orthnormée et celui de la "t-SNE" qui diminue la dimension de l'espace $\mathbb R^d$ tout en cherchant à rendre compte le mieux possible des distances de l'espace original dans le nouvel espace.


<ul>
  <li><a href="#introduction" >Introduction à la stylométrie</a></li>
  <li><a href="#logloss" >Que rechercher pour minimiser la "log loss" ?</a></li>
  <li><a href="#kppv">L'algorithme des k-plus-proches-voisins (qui minimise asymptotiquement la "log loss")</a></li>
  <li><a href="#exemples">Quelques exemples de features</a>
      <ul style="list-style-type:circle">
      <li><a href="#exemples_courants">Exemples classiques de stylométrie</a></li>
      <li><a href="#voca_auteurs">Calcul de la log-vraisemblance pour chacun des auteurs en fonction du vocabulaire</a></li>
      <li><a href="#kernel">Kernel sur la structure grammaticale des phrases</a></li>
      </ul>
  </li>
  <li><a href="#visualisation">Kernel sur la structure grammaticale des phrases</a>
        <ul style="list-style-type:circle">
      <li><a href="#ACP">l'Analyse en Composantes Principale (ACP)</a></li>
      <li><a href="#tSNE">la t-SNE</a></li>
      </ul>
  </li>
   <li> <a href="#resultats"> Résultats (score : 0,40)</a></li>
</ul>


<div id="introduction" />
# Introduction à la stylométrie
Kaggle nous propose, avec l'aide d'un corpus d'entraînement, de retrouver les auteurs (qui sont au nombre de trois : Edgar A. Poe,  Howard P. Lovecraft et Mary Shelley) d'une petite dizaine de mille de phrases.

Ce que nous demande de faire Kaggle porte un nom : la* **stylométrie***.

La *stylométrie* est cet *art* (pris dans son sens grec de *τέχνη*) à la croisée de la statistique et de la linguistique dont le but est de produire à partir de textes une*information* (qui puisse être traitée statistiquement) rendant compte du *style* de ces derniers ; c'est-à-dire qui caractériserait -- idéalement, cela s'entend... -- à la fois son auteur, mais aussi son genre, son époque, etc...
Plus pratiquement, ces statistiques portent généralement sur le vocabulaire utilisé (qui peut être affiné en repérant le sens en lequel l'auteur emploie tel mot), les catégories grammaticales (noms/pronoms, adjectifs, adverbes, verbes... à quel temps/mode ?) ainsi que la structure grammaiticale des phrases, la ponctuation, etc... 

Il faut donc bien comprendre que pour faire de la bonne stylométrie, il faut à la fois jouer sur les deux tableaux :
- celui de la linguistitique en proposant des critères qui soient les plus pertinents pour isoler le style d'un auteur,
- celui de la statistique en employant des outils mathématiques et informatiques adaptés.

Au cours de ce tutoriel, nous essayerons d'aborder ces deux aspects de la stylométrie en restant toujours le plus possible rigoureux et en justifiant (autant que faire se peut) tout ce que nous ferons.

Mais d'abord, intéressons-nous au sytème d'évaluation de Spooky :

<div id="logloss" />
# Que rechercher pour minimiser la "log loss" ?

Tout d'abord nous allons voir pourquoi le système d'évaluation "logloss" a été choisi  et quelles sont les valeurs mathématiques à rechercher pour le minimiser en espérance.
De ces valeurs à rechercher, nous en déduirons un algorithme adapté (c'est-à-dire que nous justifierons d'un point de vue mathématique) pour le calcul des probabilités $p_{ij}$ à retourner. C'est l'agorithme des k-plus proches-voisins.

Le système de notation qui a été choisi pour Spooky est celui de "log loss" (ou *entropie croisée*) :
        
$$ log loss = - \frac{1}{N}\sum_{\substack{i \in \{1,...,N\} \\ j \in \{ EAP, HPL, MWS \}}}y_{ij} \log(p_{ij})$$
où $p_{ij}$ est l'estimation que l'on doit rendre ; la probabilité selon notre programme que le texte $i$ soit de l'auteur $j$.
$y_{ij}$ est la solution : il vaut $1$ lorsque le texte $i$ est de l'auteur $j$, $0$ sinon.

Le but est de minimiser cette "log loss".

Nous allons maintenant un tout petit peu formaliser le problème d'un point de vue mathématique afin de pouvoir calculer l'espérance de la "log loss" et de trouver les valeurs optimales $p_{ij}^{opt}$ à retourner afin de minimiser cette espérance.

Supposons que nous disposions déjà de la première partie du programme : celle qui transforme une phrase en une "information" traitable par l'informatique. En l'occurrence nous supposerons que ces informations sont des vecteurs de $\mathbb R ^d$ et qu'ils sont tous tirés de façon indépendante selon une même loi $X$ admettant une densité $f$ à support compact. On suppose aussi qu'on peut décomposer cette loi $\mathcal L(X)$ selon deux étapes :

-  tout d'abord, on choisit l'auteur de la phrase selon une loi de "Bernoulli à trois issues" : 'EAP' pour Edgar A. Poe, 'HPL' pour Howard P. Lovecraft et 'MWS' pour Mary Shelley. On notera $\mathcal B(\lambda_1, \lambda_2, \lambda_3)$ cette loi (avec $\sum \lambda_j = 1$) ; $\lambda_j$ étant la proportion de phrases de l'auteur $j$.
-  En fonction de l'auteur $j$ qui est choisi à la première étape, notre phrase est tirée selon une loi de densité $f_j$ sur $\mathbb R ^d$.

![shema_bernouilli_fr](http://bourg-la-reine-echecs.fr/telechargements/shema_bernoulli_auteurs_corrige.png)

On a donc que : $f = \sum \lambda_j f_j$.

Avec le corpus d'entraînement, on obtient que : 


In [ ]:
nb_texts = len(train_df)
lambdas = (train_df.author.value_counts()/nb_texts).to_dict()
print("Lambdas :",lambdas)

- $\lambda_1 = 0.403493538995863$ 
- $\lambda_2 = 0.287808366106543$
- $\lambda_3 = 0.308698094897594$

Le lecteur attentif aura peut-être remarqué que le fichier d'exemple "sample\_submission" renvoie justement pour chaque phrase les probabilités $(\lambda_1, \lambda_2, \lambda_3)$... ce n'est pas un hasard !! C'est le retour qui minimise la "log loss" avec le plus mauvais programme qui soit : celui qui ne distingue rien et qui envoie toutes les phrases sur le vecteur nul ($\mathcal L (X) = \delta_0$ où $\delta_0$ est la mesure de "dirac" au point $0$).
Cela découle d'un résultat classique de théorie de l'information : si $(\lambda_j)_j$ et $(p_j)_j$ sont deux mesures de probabilité discrètes, alors l'entropie croisée :
$$H_\lambda(p) = \mathbb E[log loss] = -\sum_j \lambda_j \log(p_j)$$
est minimisée quand $p_j = \lambda_j$ pour tout $j$.

Nous allons maintenant généraliser ce résultat !

Étant donné une phrase transformée par notre programme en un vecteur $x$ (selon donc la loi $X$), la meilleure solution (au sens que c'est celle qui minimise l'espérance de la note finale "log loss") de probabilités  à renvoyer est le triplet :

$$ p_1^{opt}(x) = \frac{\lambda_1 f_1(x)}{\sum_j \lambda_j f_j(x)}, \ \ \ \ 
p_2^{opt}(x) = \frac{\lambda_2 f_2(x)}{\sum_j \lambda_j f_j(x)}, \ \ \ \ 
p_3^{opt}(x) = \frac{\lambda_3 f_3(x)}{\sum_j \lambda_j f_j(x)}.$$

Bon ! maintenant que cela a été dit, il n'y a plus qu'à se retrousser les manches !

L'espérance de la note "log loss" est donnée dans ce cas par la formule suivante :

$$ \mathbb E [log loss] = - \sum_j{\lambda_j \int_{\mathbb R ^d}\log(p_j(x))f_j(x) \mathrm d x}$$
où $p_j(x)$ est la probabilité que l'on renvoie pour l'auteur $j$ quand on reçoit le vecteur $x$  ; c'est notre estimation que la phrase qui a été transformé en le vecteur $x$ soit de l'auteur $j$ (on veut donc montrer que $p_j^{opt}(x)= \frac{\lambda_j f_j(x)}{\sum_k \lambda_k f_k(x)}$ est effectivement la solution optimale).

En toute généralité, le programme  renverra une probabilité qui peut s'écrire sous la forme :
$$ q_j(x)= \frac{\lambda_j g_j(x)}{\sum_k \lambda_k g_k(x)}$$
où $g_j$ est une fonction continue positive (qu'on supposera quand même de support au moins aussi grand que celui de $f_j$ car s'il existe $x$ tel que $q_j(x) = 0$ et $f_j(x) > 0$, alors $\mathbb P(logloss = \infty) >0$...). 
Quitte à ce que les $g_j$ aillent "perdre" de leur poids en dehors du support de $f$ (c'est-à-dire que si $f(x) = 0$, $x$ n'a aucune chance d'apparaître ; peu nous importe donc la valeur des $g_j$ en ces points), on peut aussi supposer que les $g_j$ sont des fonctions de densité (c'est-à-dire que : $\int_{\mathbb R^d} g_j(x) dx = 1$).

On peut prendre, par exemple, $g_j = \frac{1}{\lambda_j}f_j$ pour ne plus tenir compte de la différence du nombre de phrases entre les auteurs (ici, les $g_j$ ne sont pas des fonctions de densité puisque leur intégrale ne vaut pas $1$, mais l'on pourrait s'y ramener comme dit juste avant).
Si l'on veut des probabilités "extrêmes" (proches de $0$ ou de $1$), on pourra poser : $g_j = f_j^2 \times \frac{1}{\int f_j^2}$.
Au contraire, si l'on souhaite "nuancer" ces probabilités, on posera : $ g_j = \sqrt{f_j} \times \frac{1}{\int \sqrt f_j}$. 

Montrons à présent que le meilleur choix pour minimiser la note "log loss" est bel et bien : $p_j^{opt} = \frac{\lambda_j f_j(x)}{\sum_k \lambda_k f_k(x)}$. Notons $log loss^{opt}$ la note obtenue avec ce dernier choix et $log loss^{q_j}$ la note obtenue avec les probabilités $q_j$ définies à l'aide des fonctions $g_j$. 
On a alors :

\begin{align}
\mathbb E[log loss^{opt}] - \mathbb E[log loss^{q_j}] & = - \sum_j{\lambda_j \int_{\mathbb R ^d}\log(p_j^{opt})f_j(x) \mathrm d x} - \sum_j{\lambda_j \int_{\mathbb R ^d}\log(q_j)f_j(x) \mathrm d x}\\
 & = - \sum_j{\lambda_j \int_{\mathbb R ^d}(\log(p_j^{opt})-\log(q_j))f_j(x) \mathrm d x} \\
 & = - \sum_j{\lambda_j \int_{\mathbb R ^d}\log(\frac{\lambda_j f_j(x)}{\lambda_j g_j(x)}\frac{\sum_k \lambda_k g_k(x)}{\sum_k \lambda_k f_k(x)})f_j(x) \mathrm d x} \\
 & = - \sum_j{\lambda_j \int_{\mathbb R ^d}\log(\frac{ f_j(x)}{ g_j(x)})f_j(x) \mathrm d x} + {\int_{\mathbb R ^d}\log(\frac{\sum_k \lambda_k f_k(x)}{\sum_k \lambda_k g_k(x)})(\sum_j \lambda_j f_j(x))\mathrm d x} \\
 & = - \sum_j \lambda_j D_{KL}(f_j || g_j) + D_{KL}(\sum_k \lambda_k f_k || \sum_k \lambda_k g_k)\\
 & \text{(où $D_{KL}$ désigne la divergence de Kullback-Leibler... nous y reviendrons !)} \\
 & \le 0 \text{ par la propriété de convexité de $D_{KL}$ en les paires de fonctions ;}\\
 & \text{ici, il y en a $3$ : $(f_1,g_1)$, $(f_2, g_2)$ et $(f_3,g_3)$. On conclut que $p_j^{opt}$ est bien optimal :}\\
 \mathbb E[log loss^{opt}] & \le \mathbb E[log loss^{q_j}]
\end{align}

La divergence de Kullback-Leibler entre deux densités $p$ et $q$ sur $\mathbb R^d$ est donnée par la formule suivante :
$D_{KL}(p||q) = \int_{\mathbb R^d}\log(\frac{p(x)}{q(x)})p(x)\mathrm d x$. Pour le voir rapidement, en théorie de l'information, c'est la quantité moyenne de bits supplémentaires qu'il va falloir pour coder à partir de la source $q$ un code optimal de la source $p$. 
Nous reparlerons de la divergence de Kullback-Leibler dans la partie consacrée à la "t-SNE" (algorithme qui fonctionne justement sur une minimisation de cette même divergence lors du changement de dimension de l'espace des données).


Nous avons donc franchi vaillamment la première étape (notre propos étant de montrer qu'on peut faire de l'informatique de façon propre et rigoureuse !) : on sait maintenant que, une fois que l'on possédera notre programme qui transforme les phrases en vecteurs, la quantité à rechercher (et à renvoyer à Kaggle) est :
$$ p_j^{opt}(x) = \frac{\lambda_jf_j(x)}{\sum_k \lambda_k f_k(x)}.$$



<div id="kppv" />
# k-plus-proches-voisins (pour retrouver les probabilités $p_{ij}^{opt}$ optimales)

Nous allons montrer une manière de récupérer ces probabilités. Oh ! ce n'est certes pas un algorithme extraordinaire puisque c'est celui des plus-proches-voisins.
Néanmoins, il a cet avantage de donner asymptotiquement (lorsque $N$ le nombre de textes tend vers l'infini) le meilleur résultat possible. D'autres algorithmes souvent employés font par exemple l'hypothèse d'une distribution gaussienne des données (c'est le cas des classificateurs naïfs de Bayes) qui donnent de bons résultats sur de petits échantillons (puisqu'il se basent sur le calcul des espérances et des covariances des variables ; ce qui nécessite beaucoup moins de données que pour estimer une densité) mais ces algorithmes cessent de donner de bons résultats sitôt que la base d'apprentissage est suffisante.

Voici donc l'algorithme des k-plus-proches-voisins :

- On possède une base d'apprentissage $\{(x_1,y_1), \ ... \ , \ (x_N,y_N)\}$ contenant les $N$ vecteurs $x_1, ..., x_N \in \mathbb R ^d$ formés à partir des $N$ phrases avec leur auteur respectif $y_1,...,y_N \in \{1,2,3\}$. On choisit aussi une distance $d$ sur $\mathbb R^d$ (généralement la distance euclidienne, mais cela n'est pas obligatoire ; les résultats asymptotiques resteront vrais pour n'importe quelle distance : on dit que l'opérateur des k-plus-proches voisins est \textit{universellement} consistant).
- On choisit un entier $k$ dépendant de $N$ qui doit être petit devant $N$ (on veut : $k/N \underset{N \to \infty}{\longrightarrow} 0$) mais assez grand tout de même (il faut que : $k \underset{N\to \infty}{\longrightarrow} \infty$).
- Maintenant, à chaque fois que l'on reçoit une nouvelle phrase (en fait un nouveau vecteur $x$), on considère l'ensemble $kNN$ des $k$ indices $i_1 <... < i_k$ des points les plus proches de $x$ :
$$ kNN = \{i_1 < ... < i_k \ | \ \forall \ j \notin \{i_1,...,i_N\}, \forall \ i \in \{i_1,...,i_N\}, \ \ d(x,x_i) \le d(x,x_j)\} $$
- Enfin, on retourne des probabilités $\hat p_j(x)$ proportionnelles au nombre de plus-proches-voisins qui sont de l'auteur $j$. Ainsi :
$$\text{Pour $j \in \{1,2,3\}$, } \hat p_j(x) = \frac{1}{k} \# \{ i \in kNN \ | \ y_i = j\}$$


Nous avons donc présenté l'algorithme très simple des $k$-plus-proches-voisins qui renvoie des estimations $\hat p_j(x)$ de $p_j^{opt}(x)$. Il vérifie la propriété de "consistance faible" suivante :
$$\text{Si $k \underset{N\to \infty}{\longrightarrow} \infty$ et que $k/N \underset{N \to \infty}{\longrightarrow} 0$, alors : } \forall \ \epsilon >0,\ \mathbb P (|\hat p_j - p_j^{opt}| > \epsilon ) \underset{N\to \infty}{\longrightarrow} 0$$

Asymptotiquement, on retrouve donc bien empiriquement la valeur de $p_j^{opt}$. Cela ne nous dit rien pour autant sur la vitesse de convergence. Il existe des raffinements de cet algorithme (par exemple en ayant recours à des graphes de proximité) qui permettent d'améliorer cette vitesse de convergence.



<div id="exemples" />
# Exemples


<div id="exemples_courants" />
## Exemples courants
TODO




<div id="voca_auteurs" />
## Exemple d'utilisation du vocabulaire

Les exemples que nous avons vus dans la section précédente sont des exemples courants de stylométrie. Ce sont aussi des exemples très généraux en ce sens qu'ils ne nécessitent pas la présence d'un corpus d'entraînement : ils pourraient tout aussi bien servir dans le cas de classification *non-supervisée* (et pas seulement *supervisée* comme c'est le cas ici).

Nous allons voir maintenant un exemple de fonction qui recourt au vocabulaire utilisé dans le corpus d'entraînement. Étant donné une phrase, cette fonction retourne un vecteur de taille $3$ contenant la log-vraisemblance de l'utilisation de ce vocabulaire pour chacun des $3$ auteurs.

Découpons notre corpus d'apprentisage en deux : 
- un nouveau corpus d'apprentissage pour construire le dictionnaire (95% des données)
- un corpus de test qui nous permettra de visualiser les résultats (5% des données)



In [ ]:
N_train = len(train_df)

# découpe du train
cut = round(N_train * 0.95)

train = pd.DataFrame(train_df, index=range(cut))
test = pd.DataFrame(train_df, index=range(cut,N_train))


On commence par construire un dictionnaire contenant tous les mots apparaissant dans le corpus d'entraînement. Chaque entrée de ce dictionnaire est associée à un tableau donnant le nombre d'apparitions du mot pour chacun des différents auteurs.


In [ ]:
# construit à partir d'un corpus d'entraînement un dictionnaire des mots utilisés. Pour chaque mot, on remplit un dictionnaire qui à chaque auteur associe le nombre de fois que ce mot est apparu chez l'auteur
def voca_Authors(train):
    voca = {}
    for i,line in train.iterrows():
        words = nltk.word_tokenize(line['text'])
        for word in words:
            word = word.lower()
            if not word in voca:
                voca[word] = {auth:0 for auth in authors}
                voca[word][line['author']] = 1
            else:
                voca[word][line['author']] += 1
    return voca

print('Calcul du dictionnaire du vocabulaire.')
voca_authors = voca_Authors(train)
print('Taille du dictionnaire :',len(voca_authors),'\n20 exemples tirés de ce dictionnaire :\n')
for i,word in enumerate(voca_authors):
    if i < 20:
        print(word,voca_authors[word])

Le tout maintenant est de bien voir ce que l'on calcule.
On fera l'hypothèse assez forte (c'est-à-dire assez fausse) que :
- la longueur d'une phrase suit une loi indépendante de l'auteur (on peut regarder les histogrammes des longueurs de phrase en fonction de l'auteur à l'adresse suivante : lien et constater qu'effectivement ces 3 histogrammes sont assez proches),
- tous les mots d'une phrase sont tirés indépendamment et identiquement selon une loi ne dépendant que de l'auteur (c'est cette hypothèse qui est très forte et très fausse).

Grâce aux hypothèses d'indépendances, on peut calculer la vraisemblance que l'auteur $j$ ait écrit la phrase $s = [w_1,...,w_k]$ :
$$ L (w_1, \ ... \ , \ w_k \ | \ author = j) = \prod_{i=1}^k \mathbb P(\text{author of $w_i$ = $j$} \ | \ \text{word = $w_i$})$$
À présent, prenons comme exemple le mot "of" : on a les statistiques suivantes : 



In [ ]:
voca_authors['of']

Il y a $8539+5568+5850 = 19957$ occurrences du mot "of" dans notre corpus d'entraînement. Edgar Poe est l'auteur de 8539 d'entre elles, c'est-à-dire d'une proportion $x_1 = 8539/19957 = 0.428$. Il est aussi l'auteur d'une proportion $\lambda_1 = 0.40 < x_1$ de l'ensemble des phrases du corpus. Comme on a fait l'hypothèse que la loi sur la longueur des phrases était indépendante de l'auteur, cela signifie qu'il écrit donc davantage le mot 'of' que sa sur-représentativité dans le corpus lui permettrait ; c'est-à-dire que : $ p_1 := \mathbb P(\text{author of $'of' = 1$}) > 1/3$.
Si l'on note $ p_j = \mathbb P(\text{author of $'of'  = j$})$ et $x_j$ la proportion d'apparitions dans le corpus d'entraînement, on a la relation suivante :
$$ x_j = \frac{\lambda_j p_j}{\sum_k \lambda_k p_k}.$$

On a donc les $x_j$ et les $\lambda_j$ et l'on veut retrouver les $p_j$ pour calculer nos log-vraisemblances !

À noter que l'on peut calculer très facilement $\sum_k \lambda_k p_k$ : $$ \sum_j \frac{x_j}{\lambda_j} = \sum_j \sum_k \lambda_k p_k p_j = \sum_k \lambda_k p_k$$
... et ainsi retrouver la valeur $p_j$ que l'on recherche :
$$ p_j = \frac{x_j}{\lambda_j} \times \sum_k\frac{x_k}{\lambda_k}$$

On peut alors calculer la log-vraisemblance pour chacun des auteurs :
$$ \log L(w_1,...,w_k \ | \ author = j) = \sum_{i=1}^k \log(p_j(w_i)) $$



In [ ]:
def vect_Voca(sentence, dict = voca_authors, zero = 0.01):
    words = nltk.word_tokenize(sentence)
    ret = [0]*len(authors)
    for word in words:
        word = word.lower()
        if word in dict:
            vect = [0]*len(authors)
            nb_appar = 0
            for auth in dict[word]:
                nb_appar += dict[word][auth]
            for auth in dict[word]:
                vect[authors[auth]] = dict[word][auth]/nb_appar/lambdas[auth] # vect[j] = x_j/lambda_j = p_j / (Sum lambda_j p_j)
            s = 1/sum(vect) # s = (Sum lambda_j p_j) 
            vect = [p_j_div_sum * s for p_j_div_sum in vect] # vect[j] = p_j

            for j,p_j in enumerate(vect):
                if p_j == 0:
                    ret[j] += log(zero)
                else:
                    ret[j] += log(p_j)
    return ret


... et visualiser les résultats en queue de comète :


In [ ]:
# la matrice des phrases du corpus de texte transformées à l'aide de vect_Voca
X = []
col = [] # la couleur en fonction de l'auteur : Poe=>rouge, Lovecraft=>vert, Shelley=>bleu

# calcul de X la matrice de taille N_tests * 3
for i,line in test.iterrows():
    vect = vect_Voca(line['text'])
    X.append(vect)
    col.append(colours[line['author']])
X = np.array(X)

## L'affichage

#import plotly
#from plotly.graph_objs import Scatter, Layout, Scatter3d
fig = plt.figure(figsize=(15,15))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(X[:,0], X[:,1], X[:,2], c=col, marker='+')
#plotly.iplot({
 #   "data": [Scatter3d(x=X[:,0], y=X[:,1], z=X[:,2], mode='markers', 
 #   marker=dict(size=4,
 #       color=col, 
  #      opacity=0.7
  #  ))],
  #  "layout": Layout(title="vect_Voca", scene=dict(camera= dict(
 #   up=dict(x=0, y=0, z=1),
  #  center=dict(x=0, y=0, z=0),
  #  eye=dict(x=1.5, y=0.75, z=0.475)
#)))
#})

<div id="kernel" />
## Exemple de kernel sur les structures arborescentes des phrases
TODO

<div id="visualisation" />
# Visualisation des données


<div id="ACP" />
## L'Analyse en Composantes Principales (ACP)
TODO

<div id="tSNE" />
## La t-SNE
TODO

<div id="resultats" />
# Résultats (score : 0,40)

Score obtenu simplement avec la log-vraisemblance de l'utilisation du vocabulaire :

In [ ]:
from sklearn.neighbors import NearestNeighbors

train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

voca_authors = voca_Authors(train)
# la matrice des phrases du corpus de texte transformées à l'aide de vect_Voca
M = [] # the train matrix
Sol = [] # the solutions of the train matrix
X = [] # the test matrix


for i,line in train.iterrows():
    vect = vect_Voca(line['text'])
    M.append(vect)
    Sol.append(line['author'])
M = np.array(M)

for i,line in test.iterrows():
    vect = vect_Voca(line['text'])
    X.append(vect)
X = np.array(X)


# The kNN algorithm
adjunction = 0.025
nb_ppv = 200 # number of nearest neighbors
M_ppv = NearestNeighbors(n_neighbors=nb_ppv)
M_ppv.fit(M)
tab_kneighbors = M_ppv.kneighbors(X, return_distance=False)

Probas = [] # the return

for i,line in test.iterrows():
    p = [0]*3
    kneighbors = tab_kneighbors[i]
    for kneighbor in kneighbors:
        p[authors[Sol[kneighbor]]] += 1
    s = sum(p)
    p = [x/s for x in p]
    Probas.append(p)

# harmonisation of the probabilities
for v in Probas:
    for i,prob in enumerate(v):
        v[i] = (prob + adjunction)/(1+3*adjunction)


submission = pd.read_csv('../input/sample_submission.csv')
submission.loc[:,['EAP', 'HPL', 'MWS']] = Probas
submission.to_csv("Log_likelihoof_on_vocabulary.csv", index=False)
submission.head()